<a href="https://colab.research.google.com/github/ByungHyunPark57/Project_ORI-Team_movie_review/blob/main/%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0%EC%98%88%EC%B8%A11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

#import os
#print(os.listdir("../input"))
#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
#importing the training data
data=pd.read_csv('/content/drive/MyDrive/IMDB Dataset(영화리뷰).csv')
print(data.shape)
data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
## 결측값 확인

data.isnull().sum()

,0
review,0
sentiment,0


중복값 제거하고 토큰화 해야함
토큰화 하고 스테밍 다시

---



In [ ]:
# 중복된 행 확인
duplicates = data.duplicated()

# 중복된 행 출력
print(data[duplicates])

                                                  review sentiment
3537   Quite what the producers of this appalling ada...  negative
3769   My favourite police series of all time turns t...  positive
4391   Beautiful film, pure Cassavetes style. Gena Ro...  positive
6352   If you liked the Grinch movie... go watch that...  negative
6479   I want very much to believe that the above quo...  negative
...                                                  ...       ...
49912  This is an incredible piece of drama and power...  positive
49950  This was a very brief episode that appeared in...  negative
49984  Hello it is I Derrick Cannon and I welcome you...  negative
49986  This movie is a disgrace to the Major League F...  negative
49991  Les Visiteurs, the first movie about the medie...  negative

[418 rows x 2 columns]


In [ ]:
#Summary of the dataset
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


리뷰 고유값 49582개로 중복값 418개 있다고 해석 가능

In [ ]:
data.duplicated().sum()

418

In [ ]:
# 중복된 행의 인덱스 출력
duplicate_indices = data[data.duplicated()].index
print(duplicate_indices)

Index([ 3537,  3769,  4391,  6352,  6479,  6672,  7221,  7222,  7425,  7555,
       ...
       49361, 49595, 49695, 49781, 49871, 49912, 49950, 49984, 49986, 49991],
      dtype='int64', length=418)


In [ ]:
print(data[data.duplicated(keep=False)])

                                                  review sentiment
42     Of all the films I have seen, this one, The Ra...  negative
84     We brought this film as a joke for a friend, a...  negative
140    Before I begin, let me get something off my ch...  negative
219    Ed Wood rides again. The fact that this movie ...  negative
245    I have seen this film at least 100 times and I...  positive
...                                                  ...       ...
49912  This is an incredible piece of drama and power...  positive
49950  This was a very brief episode that appeared in...  negative
49984  Hello it is I Derrick Cannon and I welcome you...  negative
49986  This movie is a disgrace to the Major League F...  negative
49991  Les Visiteurs, the first movie about the medie...  negative

[824 rows x 2 columns]


In [ ]:
## 중복값 제거

data = data.drop_duplicates()

In [ ]:
data.duplicated().sum()

0

In [ ]:
#Summary of the dataset
data.describe()

,review,sentiment
count,49582,49582
unique,49582,2
top,One of the other reviewers has mentioned that ...,positive
freq,1,24884


똑같은 리뷰 빈도 5

In [ ]:
#sentiment count
data['sentiment'].value_counts()

,count
sentiment,
positive,24884
negative,24698


긍정 부정 25000개씩

In [ ]:
## train test로 데이터셋 분리

#split the dataset
#train dataset
train_reviews=data.review[:40000]
train_sentiments=data.sentiment[:40000]
#test dataset
test_reviews=data.review[40000:]
test_sentiments=data.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(9582,) (9582,)


○ 노이즈 텍스트 제거

In [ ]:
## 노이즈 텍스트 제거
## HTML 태그와 노이즈 텍스트를 제거

#Removing the html strips
## text에서 HTML 태그를 제거
## BeautifulSoup 라이브러리를 사용
# from bs4 import BeautifulSoup

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
## text에서 대괄호 [] 사이에 있는 모든 텍스트를 제거
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
## 텍스트에서 HTML 태그와 대괄호 안의 노이즈 텍스트를 제거
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
## 각 리뷰에서 HTML 태그와 대괄호 안의 노이즈 텍스트를 제거
data['review']=data['review'].apply(denoise_text)

<ipython-input-46-3ab4667abbbd>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
## 특수문자 제거

#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
data['review']=data['review'].apply(remove_special_characters)

토큰화하고 스테밍

In [ ]:
## 텍스트 형태소 분석

#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
data['review']=data['review'].apply(simple_stemmer)

1. 스테밍(Stemming)이란?
스테밍은 단어의 접미사나 변형된 부분을 제거하여 단어의 기본 형태(어근, stem)를 추출하는 과정

ex)
"running", "runner", "ran" → "run"
"jumps", "jumping", "jumped" → "jump"

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
## 불용어 제거

#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
#Apply function on review column
data['review']=data['review'].apply(remove_stopwords)

{'other', 'very', 'all', 'over', 'theirs', 'before', 'more', 'such', 'our', 'aren', 'by', 'from', 'mustn', 'both', 'few', 'of', 'we', 'its', "you'll", 'a', "weren't", 'until', 'only', 't', "that'll", 'any', 'an', "shouldn't", 'some', 'his', 'my', "should've", 'when', 'just', 's', 'their', "wouldn't", "aren't", 'no', 'at', 'now', 'while', 'if', 'has', 'through', 'be', 'himself', "shan't", 'against', "hadn't", 'mightn', 'where', 'had', 'which', 'did', 'should', 'isn', 'won', 'again', 'ours', 'i', 'that', 'or', 'after', 'been', 'in', 'because', 'as', 'couldn', 'these', 'into', 'between', 'me', 'this', 'he', 'do', 'with', 'hasn', 'how', 'and', 'above', 'once', "isn't", 'for', 'don', 'were', 'o', 'your', 'up', 'does', 'wasn', 'weren', 'needn', 'll', 'have', 'but', "won't", 'here', 'what', "you'd", 'on', 'further', 'haven', 'who', 'during', 'out', 'why', 'whom', 'doing', 'wouldn', 'you', 'shouldn', "you're", 've', "didn't", 're', "doesn't", 'him', 'itself', 'below', 'them', 'will', "you've",

In [ ]:
train_reviews.head(10)

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."
5,"Probably my all-time favorite movie, a story o..."
6,I sure would like to see a resurrection of a u...
7,"This show was an amazing, fresh & innovative i..."
8,Encouraged by the positive comments about this...
9,If you like original gut wrenching laughter yo...


In [ ]:
test_reviews.head(10)

,review
40000,First off I want to say that I lean liberal on...
40001,I was excited to see a sitcom that would hopef...
40002,When you look at the cover and read stuff abou...
40003,"Like many others, I counted on the appearance ..."
40004,"This movie was on t.v the other day, and I did..."
40005,Hickory Dickory Dock was a good Poirot mystery...
40006,"""One Crazy Summer"" is the funniest, craziest (..."
40007,Low-budget schlockmeister Herschell Gordon Lew...
40008,this was absolutely the most tragic pile of ci...
40009,I have never understood the appeal of this sho...


In [ ]:
##test_reviews.iloc[0]

"Parsifal (1982) Starring Michael Kutter, Armin Jordan, Robert Lloyd, Martin Sperr, Edith Clever, Aage Haugland and the voices of Reiner Goldberg, Yvonne Minton, Wolfgang Schone, Director Hans-Jurgen Syberberg.<br /><br />Straight out of the German school of film, the kind that favored tons of symbolism and Ingmar Bergmanesque surrealism, came this 1982 film of Wagner's final masterpiece- Parsifal, written to correspond with Good Friday/Easter and the consecration of the Bayreuth Opera House. This film follows the musical score and plot accurately but the manner in which it was filmed and performed is bold and avant-garde and no other Parsifal takes the crown in its bizarre cinematography. Syberberg is known for controversial films. Prior to this film he had released films about Hitler and Nazism, Richard Wagner and his personal Anti-Semitism and a documentary about Winifred Wagner, one of his grand-daughters. This film is possibly disturbing in many aspects. Parsifal (sung by Reiner G

In [ ]:
train_sentiments.head(10)

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
5,positive
6,positive
7,negative
8,negative
9,positive


In [ ]:
test_sentiments.head(10)

,sentiment
40000,negative
40001,negative
40002,negative
40003,negative
40004,negative
40005,positive
40006,positive
40007,negative
40008,negative
40009,negative


In [ ]:
## 표준화된 train 데이터 리뷰

#normalized train reviews
norm_train_reviews=data.review[:40000]
norm_train_reviews[0]
#convert dataframe to string
#norm_train_string=norm_train_reviews.to_string()
#Spelling correction using Textblob
#norm_train_spelling=TextBlob(norm_train_string)
#norm_train_spelling.correct()
#Tokenization using Textblob
#norm_train_words=norm_train_spelling.words
#norm_train_words

'one review ha mention watch 1 oz episod youll hook right thi exactli happen meth first thing struck oz wa brutal unflinch scene violenc set right word go trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

In [ ]:
## 표준화된 test 데이터 리뷰

#Normalized test reviews
norm_test_reviews=data.review[40000:]
norm_test_reviews[40006]
##convert dataframe to string
#norm_test_string=norm_test_reviews.to_string()
#spelling correction using Textblob
#norm_test_spelling=TextBlob(norm_test_string)
#print(norm_test_spelling.correct())
#Tokenization using Textblob
#norm_test_words=norm_test_spelling.words
#norm_test_words

'one crazi summer funniest craziest necessarili best movi ever seenjust one crazi scene done anoth emerg never let rest one thing anoth soundtrack great song right one scenesit also clean movi littl dirti itof cours ha stori guy wouldnt trust lunch money take challeng win peopl resourc whod want see fail seriou side parent children live dream one alway open mind weigh option thi appli parent children one crazi summer parent wrong thi alway case'